In [1]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

In [2]:
# Initialize Faker
fake = Faker('id_ID')

In [4]:
# Generate dummy data for Users
def generate_users(num_users):
    users = []
    generated_usernames = set()  # Set to store generated usernames
    
    for _ in range(num_users):
        # Generate base identifier (name)
        base_identifier = fake.name().replace('.', '').replace(',', '')  # Remove dots and commas
        
        # Select words from name with more than 2 characters
        words = [word for word in base_identifier.split() if len(word) > 2]
        
        if words:
            # Choose a random word from selected words
            selected_word = random.choice(words)
        else:
            selected_word = base_identifier  # Use full name if no suitable words found
        
        # Generate name from base identifier
        name = base_identifier.capitalize()  # Use the selected word for simplicity
        
        # Generate username from base identifier (lowercase and no spaces)
        username = selected_word.lower().replace(' ', '_')
        
        # Append random numbers until username is unique
        while username in generated_usernames:
            username += str(random.randint(1, 999))  # Append random number
        
        generated_usernames.add(username)  # Add username to set
        
        # Generate email from username (using a domain) and add random numbers
        email = f"{username}@library.com"
           
        user = {
            'User_id': _+1,
            'Name': name,
            'Username': username,
            'Email': email,
            'Phone_number': fake.phone_number(),
        }
        users.append(user)
    
    return users

# Generate dummy data for Libraries
def generate_libraries(num_libraries):
    libraries = []
    for _ in range(num_libraries):
        library = {
            'Library_id': _+1,
            'Name': fake.company(),
            'Address': fake.address()
        }
        libraries.append(library)
    return libraries

# Generate dummy data for Books
def generate_books(num_books):
    books = []
    for _ in range(num_books):
        # Generate a fake title using a pattern or structure
        title = fake.catch_phrase()  # Example usage of Faker's catch_phrase method
        
        book = {
            'Book_id': _+1,
            'Title': title,
            'Author': fake.name(),
            'ISBN': fake.isbn13()
        }
        books.append(book)
    return books

# Generate dummy data for Book_availability
def generate_book_availability(books, libraries):
    book_availabilities = []
    used_libraries = set()
    
    # Ensure each book has at least one quantity owned
    for book in books:
        library = random.choice(libraries)
        book_availability = {
            'Availability_id': len(book_availabilities) + 1,
            'Book_id': book['Book_id'],
            'Library_id': library['Library_id'],
            'Quantity_owned': random.randint(1, 10)
        }
        book_availabilities.append(book_availability)
        used_libraries.add(library['Library_id'])
    
    # Ensure each library has at least one book
    for library in libraries:
        if library['Library_id'] not in used_libraries:
            book = random.choice(books)
            book_availability = {
                'Availability_id': len(book_availabilities) + 1,
                'Book_id': book['Book_id'],
                'Library_id': library['Library_id'],
                'Quantity_owned': random.randint(1, 10)
            }
            book_availabilities.append(book_availability)
    
    # Add additional random book availabilities
    for _ in range(len(books) * len(libraries) // 2):  # Adjust the multiplier as needed
        book_availability = {
            'Availability_id': len(book_availabilities) + 1,
            'Book_id': random.choice(books)['Book_id'],
            'Library_id': random.choice(libraries)['Library_id'],
            'Quantity_owned': random.randint(1, 10)
        }
        book_availabilities.append(book_availability)
    
    return book_availabilities

# Generate dummy data for Borrow ensuring Quantity_owned is updated
def generate_borrows(num_borrows, users, books, libraries, book_availabilities):
    borrows = []
    for _ in range(num_borrows):
        available_books = [ba for ba in book_availabilities if ba['Quantity_owned'] > 0]
        if not available_books:
            break  # Exit if no books with Quantity_owned > 0 are available

        book_availability = random.choice(available_books)
        borrow_date = fake.date_this_year()
        due_date = borrow_date + timedelta(days=14)
        
        borrow = {
            'Borrow_id': _ + 1,
            'User_id': random.choice(users)['User_id'],
            'Book_id': book_availability['Book_id'],
            'Library_id': book_availability['Library_id'],
            'Borrow_date': borrow_date,
            'Due_date': due_date
        }
        borrows.append(borrow)
        
        # Update the Quantity_owned
        book_availability['Quantity_owned'] -= 1
        
    return borrows

# Generate dummy data for Return based on Borrows
def generate_returns(borrows):
    returns = []
    return_id = 1 
    
    for borrow in borrows:
        if random.choice([True, False]):
            return_entry = {
                'Return_id': return_id,
                'Borrow_id': borrow['Borrow_id'],
                'Return_date': fake.date_between(start_date=borrow['Borrow_date'], end_date=borrow['Due_date'])
            }
            returns.append(return_entry)
            return_id += 1 
    return returns

# Generate dummy data for Hold ensuring it only includes books with Quantity_owned = 0
def generate_holds(num_holds, users, books, libraries, book_availabilities):
    holds = []
    zero_quantity_books = [ba for ba in book_availabilities if ba['Quantity_owned'] == 0]
    
    for _ in range(num_holds):
        if not zero_quantity_books:
            break  # Exit if no books with Quantity_owned = 0 are available
        hold_date = fake.date_this_year()
        zero_quantity_book = random.choice(zero_quantity_books)
        hold = {
            'Hold_id': _ + 1,
            'User_id': random.choice(users)['User_id'],
            'Book_id': zero_quantity_book['Book_id'],
            'Library_id': zero_quantity_book['Library_id'],
            'Hold_date': hold_date,
        }
        holds.append(hold)
    return holds

In [6]:
# Generate data
num_users = 200
num_libraries = 20
num_books = 200
num_borrows = 100
num_holds = 50

users = generate_users(num_users)
libraries = generate_libraries(num_libraries)
books = generate_books(num_books)
book_availabilities = generate_book_availability(books, libraries)
borrows = generate_borrows(num_borrows, users, books, libraries, book_availabilities)
returns = generate_returns(borrows)
holds = generate_holds(num_holds, users, books, libraries, book_availabilities)

In [7]:
# Convert data to DataFrames
df_users = pd.DataFrame(users)
df_libraries = pd.DataFrame(libraries)
df_books = pd.DataFrame(books)
df_book_availability = pd.DataFrame(book_availabilities)
df_borrows = pd.DataFrame(borrows)
df_returns = pd.DataFrame(returns)
df_holds = pd.DataFrame(holds)

In [8]:
# Save DataFrames to CSV files
df_users.to_csv('users.csv', index=False)
df_libraries.to_csv('libraries.csv', index=False)
df_books.to_csv('books.csv', index=False)
df_book_availability.to_csv('book_availability.csv', index=False)
df_borrows.to_csv('borrows.csv', index=False)
df_returns.to_csv('returns.csv', index=False)
df_holds.to_csv('holds.csv', index=False)

In [ ]:
-- Copy data from CSV files into PostgreSQL tables
COPY Users FROM '/path/to/your/csv/users.csv' DELIMITER ',' CSV HEADER;
COPY Libraries FROM '/path/to/your/csv/libraries.csv' DELIMITER ',' CSV HEADER;
COPY Books FROM '/path/to/your/csv/books.csv' DELIMITER ',' CSV HEADER;
COPY Book_availability FROM '/path/to/your/csv/book_availability.csv' DELIMITER ',' CSV HEADER;
COPY Requests FROM '/path/to/your/csv/requests.csv' DELIMITER ',' CSV HEADER;
COPY Borrow FROM '/path/to/your/csv/borrows.csv' DELIMITER ',' CSV HEADER;
COPY Return FROM '/path/to/your/csv/returns.csv' DELIMITER ',' CSV HEADER;
COPY Hold FROM '/path/to/your/csv/holds.csv' DELIMITER ',' CSV HEADER;
COPY Queue FROM '/path/to/your/csv/queues.csv' DELIMITER ',' CSV HEADER;